In [346]:
import gym
import numpy as np
from matplotlib import pyplot
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from collections import defaultdict
from functools import partial

env = gym.make("Blackjack-v1", sab=True)
print(env.reset())
print(env.action_space.n)

((8, 9, False), {})
2


In [347]:
print(env.step(0))
def policy_function(state):
    total_sum = 0
    for i in range(len(state[0]) - 1):
        total_sum += state[0][i]
    if total_sum > 19:
        return 0
    else:
        return 1

((8, 9, False), -1.0, True, False, {})


c:\Python311\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [348]:
def generate_episode():
    num_timesteps = 100
    states, actions, rewards = [], [], []
    state = env.reset()
    for i in range(num_timesteps):
        action = policy_function(state)
        next_state, reward, done, info, comm = env.step(action)
        states.append(next_state)
        actions.append(action)
        rewards.append(reward)
        if done == True:
            break
    return states, actions, rewards

In [349]:
def mc_every_visit(num_iterations):
    total_return = defaultdict(float)
    N = defaultdict(int)
    value_table = defaultdict(lambda: [0, []])  # Structure de value_table: {S: [total_return/N, [(action, reward), ...]]}

    for i in range(num_iterations):
        states, actions, rewards = generate_episode()
        visited_states = set()

        for t in range(len(states)):
            S = states[t]
            A = actions[t]
            R = rewards[t]

            if S not in visited_states:
                visited_states.add(S)
                total_return[S] += sum(rewards[t:])
                N[S] += 1
                value_table[S][1].append((A, R))  # Ajouter l'action et la récompense à la liste pour l'état S
                value_table[S][0] = total_return[S] / N[S] if N[S] > 0 else 0

    return value_table

In [350]:
value_ev = mc_every_visit(100000)

In [351]:
for i in range(100):
    print(value_ev.popitem())

((6, 4, False), [-1.0, [(1, 0.0)]])
((6, 9, False), [-1.0, [(1, 0.0)]])
((6, 1, False), [-1.0, [(1, 0.0), (1, 0.0), (1, 0.0)]])
((13, 3, True), [-1.0, [(1, 0.0), (1, 0.0)]])
((6, 8, False), [-1.0, [(1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0)]])
((7, 2, False), [-1.0, [(1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0)]])
((6, 3, False), [-1.0, [(1, 0.0), (1, 0.0), (1, 0.0)]])
((13, 1, True), [-1.0, [(1, 0.0), (1, 0.0), (1, 0.0)]])
((6, 5, False), [-1.0, [(1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0)]])
((13, 5, True), [-1.0, [(1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0)]])
((6, 7, False), [-1.0, [(1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0)]])
((8, 4, False), [-1.0, [(1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0)]])
((7, 4, False), [-1.0, [(1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 

In [352]:
def plot_blackjack(V, ax1, ax2):
    player_sum = np.arange(12, 21 + 1)
    dealer_show = np.arange(1, 10 + 1)
    usable_ace = np.array([False, True])
    state_values = np.zeros((len(player_sum), len(dealer_show), len(usable_ace)))

    for i, player in enumerate(player_sum):
        for j, dealer in enumerate(dealer_show):
            for k, ace in enumerate(usable_ace):
                print([i,player], [j,dealer], [k,ace])
                state_values[i, j, k] = V[player, dealer, ace]
    
    X, Y = np.meshgrid(player_sum, dealer_show)
 
    ax1.plot_wireframe(X, Y, state_values[:, :, 0])
    ax2.plot_wireframe(X, Y, state_values[:, :, 1])
 
    for ax in ax1, ax2:
        ax.set_zlim(-1, 1)
        ax.set_ylabel('player sum')
        ax.set_xlabel('dealer showing')
        ax.set_zlabel('state-value')

In [353]:
def mc_first_visit(num_iterations):
    value_table = defaultdict(lambda: [0.0, []])  # Initialiser value_table avec une liste pour chaque clé
    N = defaultdict(int)
    actions_table = defaultdict(list)

    for i in range(num_iterations):
        states, actions, rewards = generate_episode()
        returns = 0
        visited_states = set()
        print(states)
        for t in range(len(states) - 1, -1, -1):
            S = states[t]
            A = actions[t]
            R = rewards[t]

            returns += R
            if S not in visited_states:
                visited_states.add(S)
                N[S] += 1
                value_table[S][0] += (returns - value_table[S][0]) / N[S]  # Mise à jour de la valeur pour l'état S
                actions_table[S].append(A)
                value_table[S][1] = actions_table[S]  # Mettre à jour les actions pour l'état S

    return value_table

In [354]:

value_fv= mc_first_visit(50000)



[(14, 7, False)]
[(13, 6, True), (19, 6, True), (13, 6, False), (15, 6, False), (25, 6, False)]
[(16, 10, False)]
[(12, 9, False)]
[(20, 3, False)]
[(15, 8, False)]
[(18, 5, True)]
[(20, 9, False)]
[(22, 6, False)]
[(12, 8, False)]
[(24, 5, False)]
[(20, 10, False)]
[(13, 9, False)]
[(14, 4, False), (18, 4, False), (20, 4, False), (22, 4, False)]
[(17, 6, False), (27, 6, False)]
[(11, 9, False), (20, 9, False), (21, 9, False), (22, 9, False)]
[(22, 2, False)]
[(20, 5, True)]
[(17, 10, True)]
[(19, 5, True)]
[(21, 9, True), (17, 9, False), (27, 9, False)]
[(18, 10, False)]
[(20, 5, False)]
[(16, 7, False)]
[(15, 10, True)]
[(19, 7, True)]
[(21, 10, True)]
[(24, 5, False)]
[(26, 1, False)]
[(13, 7, False), (23, 7, False)]
[(12, 10, False)]
[(14, 10, False)]
[(20, 10, True)]
[(19, 10, False)]
[(20, 3, False)]
[(21, 9, True)]
[(18, 1, False), (20, 1, False), (21, 1, False), (30, 1, False)]
[(20, 9, True)]
[(20, 6, False)]
[(16, 2, False), (19, 2, False), (24, 2, False)]
[(11, 10, False)]
[

In [355]:
for i in range(100):
    print(value_fv.popitem())

((13, 2, True), [-1.0, [1, 1]])
((7, 4, False), [-1.0, [1, 1, 1]])
((13, 1, True), [-1.0, [1, 1]])
((6, 1, False), [-1.0, [1]])
((7, 7, False), [-1.0, [1, 1, 1, 1, 1, 1, 1, 1]])
((6, 5, False), [-1.0, [1, 1]])
((13, 4, True), [-1.0, [1, 1, 1]])
((7, 2, False), [-1.0, [1, 1, 1]])
((6, 7, False), [-1.0, [1]])
((6, 9, False), [-1.0, [1]])
((6, 3, False), [-1.0, [1]])
((6, 8, False), [-1.0, [1]])
((6, 6, False), [-1.0, [1, 1, 1, 1]])
((7, 9, False), [-1.0, [1, 1, 1, 1, 1]])
((7, 1, False), [-1.0, [1, 1, 1, 1, 1, 1]])
((7, 5, False), [-1.0, [1, 1, 1, 1, 1, 1, 1]])
((8, 1, False), [-1.0, [1, 1, 1, 1, 1, 1, 1, 1, 1]])
((14, 2, True), [-1.0, [1, 1, 1]])
((9, 7, False), [-1.0, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
((8, 3, False), [-1.0, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
((9, 9, False), [-1.0, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
((10, 3, False), [-1.0, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
((7, 3, False

In [416]:
def epsilon_greedy_policy(state, Q_values, epsilon):
    random_num = np.random.uniform(0, 1)
    action = None  # Default value for action
    
    if isinstance(state[0], float):
        print("Do nothing")
    else: 
        state = state[0]
    if state not in Q_values or Q_values[state] == [0.0, []]:
        return epsilon_greedy_policy(env.reset(), Q_values, epsilon)
    else:
        actions = Q_values[state]
        if random_num < epsilon:
            # Choose a random action among possible actions
            action_index = np.random.randint(0, len(actions[1]))       
            action = actions[1][action_index]  # Select a random action
        else:
            # Choose the action with the highest Q-value
            action = max(actions[1])  # Select action with the highest reward
    
    return action



In [417]:
def generate_episode(Q, num_timesteps, epsilon):
    states, actions, rewards = [], [], []
    state = env.reset()
    
    for i in range(num_timesteps):
        action = epsilon_greedy_policy(state, Q, epsilon)
        next_state, reward, done, info, comm = env.step(action)
        
        states.append(state)
        actions.append(action)
        rewards.append(reward)
        
        if done:
            break
        
        state = next_state
    
    return states, actions, rewards


In [418]:
def on_policy(Q_values, num_iteration):
    total_return = defaultdict(float)
    N = defaultdict(int)
    if num_iteration != 0:
        states, actions, rewards=generate_episode(Q_values, 100, 0.1)
        for t in range(len(states) - 1, -1, -1):
            return_r = defaultdict(float)
            S = states[t]
            A = actions[t]
            R = rewards[t]
            state_action = S[0],A #[0] pour prendre uniquement l'état et pas les écolades vides

            return_r[state_action] += sum(rewards[t:])
            total_return[state_action] += total_return[state_action]+return_r[state_action]
            N[state_action] += 1
            Q_values[state_action] = total_return[state_action]/N[state_action]
        return on_policy(Q_values, num_iteration-1)
    else:
        return dict(Q_values)
        

In [423]:
value_op=on_policy(value_fv, 2000)
for i in range(500):
    print(value_op.popitem())

(((14, 9, True), 0), 1.0)
(((20, 1, True), 0), 1.0)
(((4, 7, False), 0), 1.0)
(((16, 3, True), 0), -1.0)
(((4, 7, False), 1), -1.0)
(((4, 8, False), 0), 1.0)
(((20, 9, True), 0), 1.0)
(((18, 2, False), 0), 1.0)
(((5, 8, False), 0), -1.0)
(((15, 2, True), 0), -1.0)
(((19, 1, True), 0), 1.0)
(((20, 8, True), 0), -1.0)
(((12, 2, True), 0), -1.0)
(((5, 4, False), 0), -1.0)
(((14, 2, True), 0), -1.0)
(((13, 2, True), 0), -1.0)
(((20, 7, True), 0), 1.0)
(((15, 5, True), 0), 1.0)
(((18, 8, True), 0), -1.0)
(((7, 7, False), 0), -1.0)
(((17, 6, True), 0), -1.0)
(((7, 3, False), 0), 1.0)
(((16, 1, True), 0), -1.0)
(((13, 9, True), 0), -1.0)
(((19, 6, True), 0), 1.0)
(((6, 5, False), 0), -1.0)
(((18, 7, True), 0), 1.0)
(((6, 9, False), 0), -1.0)
(((17, 6, False), 0), -1.0)
(((6, 3, False), 0), 1.0)
(((16, 5, True), 0), -1.0)
(((19, 3, True), 0), 1.0)
(((15, 7, False), 0), -1.0)
(((10, 6, False), 0), 1.0)
(((6, 8, False), 0), -1.0)
(((20, 4, True), 0), 1.0)
(((8, 8, False), 0), -1.0)
(((18, 9, Tru